<a href="https://colab.research.google.com/github/den-vasyliev/aire/blob/demo/litellm_tracing_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing and Evaluating a LangChain OpenAI Agent Application</h1>

With the new OpenAI API that supports function calling, it’s never been easier to build your own agent.

In this notebook tutorial, we showcase how to write your own OpenAI agent in under 50 lines of code and use Phoenix to inspect the internals of the Agent. It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

## 1. Install Dependencies and Import Libraries

Install Phoenix, LangChain, and OpenAI.

In [37]:
!pip install arize-phoenix
!pip install openinference-instrumentation-litellm "litellm>=1"

  Using cached openinference_instrumentation_litellm-0.1.11-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 29.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


Import libraries.

In [43]:
import os
from getpass import getpass

import openai
import phoenix as px
import phoenix as px
from phoenix.otel import register
import litellm

## 2. Launch Phoenix

You can run Phoenix in the background to collect trace data emitted by any LangChain application that has been instrumented with the `OpenInferenceTracer`.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run a LangChain application).

In [39]:
px.launch_app()

🌍 To view the Phoenix app in your browser, visit https://2d7e1b7cp6x5-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [40]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key = userdata.get('OPENAI_API_KEY')

## 3. Configure Your OpenAI Credentials

Let's make sure we have openAI credentials set up.

In [41]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

## 4. Build and Instrument Your Agent

Let's now instrument LangChain to send trace data to Phoenix.

In [42]:
# configure the Phoenix tracer
tracer_provider = register(
  project_name="my-llm-app", # Default is 'default'
  auto_instrument=True # Auto-instrument your app based on installed OI dependencies
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: my-llm-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



Let's now define the LLM model we will use for our agent.

Let's setup the Prompt Template. This will inform how the agent will respond to queries.

In [45]:
completion_response = litellm.completion(model="gpt-4-0613",
                   messages=[{"content": "What's the capital of China?", "role": "user"}])
print(completion_response)

ModelResponse(id='chatcmpl-BAn6D2ArZIoU8tsDZdf89VtBav7fS', created=1741911717, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='The capital of China is Beijing.', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None, 'annotations': []}))], usage=Usage(completion_tokens=8, prompt_tokens=14, total_tokens=22, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')


Now, we define our agent that’s capable answering questions and calling tools.

The meat of the agent logic breaks down into three steps:

- Call OpenAI to decide which tool (if any) to call and with what arguments.

- Call the tool with the arguments to obtain an output

- Call OpenAI to synthesize a response from the conversation context and the tool output.

Let's initialize the agent.

## 5. Chat With Your Agent

Let's now chat with our agent! Note that the `OpenInferenceTracer` is logging traces of the execution to phoenix as you invoke the agent!

Let's chat with our agent a few more times. This time with some follow-up questions.

Open the `session.url` in your browser to take a look at the traces in Phoenix. Note that LLM spans contain the OpenAI function calls, and that we can inspect what tool the LLM picked based on the queries.

To learn more about function calling, check out the [OpenAI API docs](https://openai.com/blog/function-calling-and-other-api-updates).


In [46]:
print(f"View the traces in phoenix: {session.url}")

View the traces in phoenix: https://2d7e1b7cp6x6-496ff2e9c6d22116-6006-colab.googleusercontent.com/


## 6. Export Your Trace Data

You can export your trace data as a pandas dataframe for further analysis and evaluation.

In [47]:
trace_df = px.Client().get_spans_dataframe()
trace_df

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id
context.span_id,,,,,,,,,,


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/data_table.py", line 187, in _repr_mimebundle_
    dataframe = self._preprocess_dataframe()
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/colab/data_table.py", line 180, in _preprocess_dataframe
    dataframe = dataframe.reset_index()
                ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/frame.py", line 6472, in reset_index
    new_obj.insert(
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/frame.py", line 5158, in insert
    raise ValueError(f"cannot insert {column}, already exists")
ValueError: cannot insert context.span_id, already exists
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/data_table.py", line 199, in _repr_javascript_module_
    return self._gen_js(self._preprocess_dataframe())
                        ^^^^^^^^^^^^^^^^